In [16]:
import pandas as pd
from text_mining import TextMining

### Importer le CSV

In [17]:
df = pd.read_csv('../data/train_tweets.csv')

### Traitement des valeurs nuls ou non pertinentes

In [18]:
df.drop_duplicates(inplace=True)


In [19]:
print(df.shape)

(7613, 5)


In [20]:
df.dropna(inplace=True)
print(df.shape) 
df.info()

(5080, 5)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5080 entries, 31 to 7581
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        5080 non-null   int64 
 1   keyword   5080 non-null   object
 2   location  5080 non-null   object
 3   text      5080 non-null   object
 4   target    5080 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 238.1+ KB


### Gestion  des valeurs manquantes

Au lieu de supprimer 33% des données avec `dropna()`, considérons cette approche :

In [21]:
# # Alternative recommandée : imputation intelligente
# df_alternative = df.copy()

# # 1. Remplacer les valeurs manquantes par des chaînes vides ou des valeurs par défaut
# df_alternative['location'] = df_alternative['location'].fillna("unknown_location")
# df_alternative['keyword'] = df_alternative['keyword'].fillna("no_keyword")

# print(f"Données conservées : {df_alternative.shape[0]} lignes (vs {df.shape[0]} avec dropna)")
# print(f"Pourcentage conservé : {df_alternative.shape[0]/7613*100:.1f}%")


In [22]:
# Alternative recommandée : imputation intelligente
df_alternative = df.copy()

# 1. Remplacer les valeurs manquantes par des chaînes vides ou des valeurs par défaut
def replace_missing_values(series, default_value):
    df_alternative[series] = df_alternative[series].fillna(default_value)

    print(f"Données conservées de {series} : {df_alternative.shape[0]} lignes (vs {df.shape[0]} avec dropna)")
    print(f"Pourcentage conservé : {df_alternative.shape[0]/7613*100:.1f}%")

replace_missing_values('location', "")
replace_missing_values('keyword', "")

Données conservées de location : 5080 lignes (vs 5080 avec dropna)
Pourcentage conservé : 66.7%
Données conservées de keyword : 5080 lignes (vs 5080 avec dropna)
Pourcentage conservé : 66.7%


In [23]:
df.head()

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0


In [24]:
df['target'].value_counts(normalize=True)

0    0.567717
1    0.432283
Name: target, dtype: float64

### Proportion target

In [25]:

df_0 = df[df['target'] == 0]
df_1 = df[df['target'] == 1]

# 2. Échantillonner aléatoirement la classe 0 pour avoir autant d'exemples que la classe 1
df_0_sampled = df_0.sample(n=len(df_1), random_state=42)

df_balanced = pd.concat([df_0_sampled, df_1], axis=0).sample(frac=1, random_state=42)  # shuffle le tout


In [26]:

df_balanced['target'].value_counts(normalize=True)

1    0.5
0    0.5
Name: target, dtype: float64

In [27]:
df_balanced['keyword'].value_counts()


collision                33
outbreak                 31
sandstorm                31
fatalities               30
emergency%20plan         29
                         ..
razed                     9
detonation                8
epicentre                 7
radiation%20emergency     6
inundation                3
Name: keyword, Length: 221, dtype: int64

### Nettoyage

- Conversion en minuscules
- Suppression des URL
- mentions et hashtags
- Suppression de la ponctuation et caractères spéciaux
- Tokenisation
- Suppression des stopwords
- Lemmatisation (ou stemming)
- Vectorisation des textes

In [28]:
data_clean = df.copy()
tm = TextMining(data_clean)
data_clean = (
    tm.lowercase()
        .extract_target_char("#", "hashtags")
        .extract_target_char("@", "mentions")
        .extract_url()
        .clean_regex()
        .tokenize()
        .remove_stopwords()
        .apply_lemmatizer() # .apply_lemmatizer() ou .apply_stemmer()
        .vectorize(mode="tfidf") # ou mode="bow" ou mode="tfidf"
        .get_df()   
)
tm.export_csv("lemmatizer_tfidf.csv")
data_clean.head(10)


,id,keyword,location,text,target,hashtags,mentions,urls,tokens,vector
31,48,ablaze,birmingham,wholesale markets ablaze,1,,bbcmtd,http t co lhyxeohy6c,"[wholesale, market, ablaze]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
32,49,ablaze,est september 2012 bristol,we always try to bring the heavy,0,metal rt,,http t co yao1e0xngw,"[always, try, bring, heavy, metal, rt]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
33,50,ablaze,africa,breaking news nigeria flag set ablaze in aba,1,africanbaze,,http t co 2nndbgwyei,"[breaking, news, nigeria, flag, set, ablaze, a...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
34,52,ablaze,philadelphia pa,crying out for more set me ablaze,0,,,,"[cry, set, ablaze]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
35,53,ablaze,london uk,on plus side look at the sky last night it was...,0,,,http t co qqsmshaj3n,"[plus, side, look, sky, last, night, ablaze]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
36,54,ablaze,pretoria,they ve built so much hype around new acquisit...,0,mufc,phdsquares,,"[built, much, hype, around, new, acquisition, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
37,55,ablaze,world wide,inec office in abia set ablaze,1,,,http t co 3imaomknna,"[inec, office, abia, set, ablaze]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
39,57,ablaze,paranaque city,ablaze for you lord d,0,,,,"[ablaze, lord]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
40,59,ablaze,live on webcam,check these out,0,nsfw,,http t co roi2nsmejj http t co 3tj8zjin21 ht...,"[check, nsfw]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
42,62,ablaze,milky way,had an awesome time visiting the cfc head offi...,0,,,,"[awesome, time, visiting, cfc, head, office, a...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [29]:
df.head()

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0


### Validation du nettoyage

Toujours vérifier l'impact du preprocessing :

In [30]:
# Validation des résultats de nettoyage
print("🔍 Validation du nettoyage")
print("=" * 50)

# 1. Vérifier l'équilibrage des classes
print("Distribution des classes après équilibrage :")
print(df['target'].value_counts(normalize=True))
print()

# 2. Comparer longueurs avant/après nettoyage
print(f"Longueur moyenne avant nettoyage : {df['text'].str.len().mean():.1f} caractères")
print(f"Longueur moyenne après nettoyage : {data_clean['text'].str.len().mean():.1f} caractères")
print()

# 3. Exemples de transformation
print("📝 Exemples de nettoyage :")
for i in range(3):
    original = df['text'].iloc[i]
    cleaned = data_clean['text'].iloc[i]
    print(f"\nOriginal: {original}")
    print(f"Nettoyé: {cleaned}")
    print("-" * 30)

# 4. Statistiques finales
print(f"\n Dataset final :")
print(f"- Nombre de lignes : {len(df)}")
print(f"- Équilibrage : {df['target'].value_counts().to_dict()}")

🔍 Validation du nettoyage
Distribution des classes après équilibrage :
0    0.567717
1    0.432283
Name: target, dtype: float64

Longueur moyenne avant nettoyage : 102.4 caractères
Longueur moyenne après nettoyage : 78.1 caractères

📝 Exemples de nettoyage :

Original: @bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C
Nettoyé: wholesale markets ablaze
------------------------------

Original: We always try to bring the heavy. #metal #RT http://t.co/YAo1e0xngw
Nettoyé: we always try to bring the heavy 
------------------------------

Original: #AFRICANBAZE: Breaking news:Nigeria flag set ablaze in Aba. http://t.co/2nndBGwyEi
Nettoyé:   breaking news nigeria flag set ablaze in aba 
------------------------------

 Dataset final :
- Nombre de lignes : 5080
- Équilibrage : {0: 2884, 1: 2196}
